In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
train_csv_path = "../input/cassava-leaf-disease-classification/train.csv"
label_json_path = "../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"
images_dir_path = "../input/cassava-leaf-disease-classification/train_images"

In [ ]:
train_csv = pd.read_csv(train_csv_path)
train_csv['label'] = train_csv['label'].astype('string')

label_class = pd.read_json(label_json_path, orient='index')
label_class = label_class.values.flatten().tolist()
print("Label names :")
for i, label in enumerate(label_class):
    print(f" {i}. {label}")

In [ ]:
train_csv.head()

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = 100

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split = 0.2
)

valid_datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split = 0.2
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_csv,
    directory = images_dir_path,
    x_col = "image_id",
    y_col = "label",
    target_size = (IMG_SIZE, IMG_SIZE),
    class_mode = "categorical",
    batch_size = BATCH_SIZE,
    shuffle = True,
    subset = 'training'
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe = train_csv,
    directory = images_dir_path,
    x_col = "image_id",
    y_col = "label",
    target_size = (IMG_SIZE,IMG_SIZE),
    class_mode = "categorical",
    batch_size = BATCH_SIZE,
    shuffle = False,
    subset = 'validation'
)

In [ ]:
model = Sequential()
model.add(Conv2D(64,(5,5),input_shape = (IMG_SIZE,IMG_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(128,(5,5),padding = 'same',strides = (2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
#model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3),padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3),padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))

# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))

# model.add(Dense(64))
# model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(loss = 'binary_crossentropy',
             optimizer='adam',
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, 
                           patience=3, mode='min', verbose=1)
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,'chkpt_{epoch}')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                         save_weights_only=True)

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100,callbacks = [early_stop,checkpoint_callback])

In [ ]:
preds = []
ss = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

for image in ss.image_id:
    img = tf.keras.preprocessing.image.load_img('../input/cassava-leaf-disease-classification/test_images/' + image)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
    prediction = model.predict(img/255)
    preds.append(np.argmax(prediction))

my_submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
my_submission.to_csv('submission.csv', index=False) 

In [ ]:
print("Submission File: \n---------------\n")
print(my_submission.head())